In [1]:
# set root directory
import os

os.chdir('../')
os.getcwd()

'c:\\Users\\HP\\Desktop\\clustering-moroccan-weather-data'

In [2]:
# imports
import numpy as np
import pandas as pd

In [3]:
# imports for plots
from plotly import graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots
import plotly.io as pio

pio.templates.default = "plotly_white"

In [4]:
# load morocco cities
morocco_cities = pd.read_csv("processed_data/moroccan_cities.csv")

In [5]:
# pick randomly 50 moroccan cities
nb_locations = 50

np.random.seed(11111111)
arr_index = np.random.random_integers(0, len(morocco_cities), nb_locations)

arr_index

<ipython-input-5-09d49e779fbd>:5: DeprecationWarning: This function is deprecated. Please call randint(0, 698 + 1) instead
  arr_index = np.random.random_integers(0, len(morocco_cities), nb_locations)


array([525, 332, 150, 667, 486, 352, 228, 628, 528, 382, 696,  52, 272,
       377, 131, 175, 566, 183, 655, 168, 388, 589, 361, 473,  95, 294,
       437, 217, 600, 656, 171, 642, 497, 225, 125, 469, 679, 510, 674,
       565,  33, 229, 262, 455, 308, 514, 490, 623, 494, 590])

In [6]:
len(set(arr_index))

50

In [7]:
weather_var = "cumulative_PRECTOT"

In [9]:
# # perform uni and multi variate clustering on these locations
from py_scripts.fetch import LocationWeather
from py_scripts.process import process_data
from py_scripts.clustering import univariate_clustering, multivariate_clustering

for i in list(arr_index) + [36, 19]: # add Sidi Sliman and Beni Mellal
    lat = morocco_cities["lat"][i]
    lon = morocco_cities["lon"][i]
    city_name = morocco_cities["city_name"][i]

    # fetch and process data
    raw_data = LocationWeather(lat, lon).weather_data_frame
    processed_data = process_data(raw_data)

    # uni and mutli var clustering
    dict_uni = univariate_clustering(processed_data, weather_var)
    dict_multi = multivariate_clustering(processed_data)

    dict_uni_multi_clusters = { "uni": dict_uni, "multi": dict_multi }

    # save dict
    np.save(f"data_uni_multi_cluster_morocco/{city_name}.npy", dict_uni_multi_clusters)

In [11]:
# load data uni multi var clustering for each location
import os

dir_name = "data_uni_multi_cluster_morocco/"
dict_location_data = {}

for filename in os.listdir(dir_name):
    location_name = filename.split(".")[0]
    path = dir_name + filename

    dict_location_data[location_name] = np.load(path, allow_pickle='TRUE').item()

In [12]:
dict_location_data['Douar Souk L`qolla']["uni"]["High"].drop(dict_location_data['Douar Souk L`qolla']["multi"]["High"])

Index(['1990', '1991', '1997', '1998', '2001', '2003', '2004', '2009', '2011',
       '2013', '2018'],
      dtype='object')

In [13]:
# to check if clusters are identical
# def is_identical(data_region, cluster_name):
#     try:
#         data_region["uni"][cluster_name].drop(data_region["multi"][cluster_name])
#     except:
#         return False

#     return True

def is_identical(data_region, cluster_name):
    max_n = max(len(data_region["uni"][cluster_name]), len(data_region["multi"][cluster_name]))
    inter_n = len(data_region["uni"][cluster_name].intersection(data_region["multi"][cluster_name]))

    return (max_n - inter_n) <= 2

is_identical(dict_location_data['Afourar'], "Medium")

True

In [14]:
dict_prec_dominence = {}

for city_name, data_city in dict_location_data.items():
    if False in [is_identical(data_city, cluster_name=label) for label in ["High", "Medium", "Low"]]:
        dict_prec_dominence[city_name] = 0
    else:
        dict_prec_dominence[city_name] = 1

In [15]:
dict_prec_dominence.values()

dict_values([1, 0, 1, 1, 0, 1, 1, 1, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 1, 1, 0, 0, 0, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 0, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 1])

In [ ]:
# # check for Sidi Slimane

# data = morocco_cities[morocco_cities["city_name"] == "Beni Mellal"]

# lat = data["lat"].values[0]
# lon = data["lon"].values[0]
# city_name = data["city_name"].values[0]

# # fetch and process data
# raw_data = LocationWeather(lat, lon).weather_data_frame
# processed_data = process_data(raw_data)

# # uni and mutli var clustering
# dict_uni = univariate_clustering(processed_data, weather_var)
# dict_multi = multivariate_clustering(processed_data)

# dict_uni_multi_clusters = { "uni": dict_uni, "multi": dict_multi }

# # save dict
# np.save(f"data_uni_multi_cluster_morocco/{city_name}.npy", dict_uni_multi_clusters)

In [ ]:
# is_identical(dict_uni_multi_clusters, "Low")

(20, 8)

In [ ]:
# print(len(dict_uni_multi_clusters["multi"] ["Low"]) + len(dict_uni_multi_clusters["multi"] ["Medium"]) + len(dict_uni_multi_clusters["multi"] ["High"]))


39


In [20]:
print(dict_location_data["Beni Mellal"]["multi"]["Low"])
print(dict_location_data["Beni Mellal"]["uni"]  ["Low"])

Index(['1983', '1984', '1985', '1986', '1992', '1993', '1995', '1998', '1999',
       '2000', '2001', '2002', '2004', '2005', '2007', '2008', '2014', '2016',
       '2017', '2018', '2019', '2020'],
      dtype='object')
Index(['1983', '1984', '1985', '1986', '1992', '1993', '1995', '1999', '2000',
       '2002', '2005', '2007', '2014', '2016', '2017', '2018', '2019', '2020'],
      dtype='object')


In [ ]:
# data = dict_location_data["Sidi Slimane"]

# print(data["uni"]["Low"])
# print(data["multi"]["Low"])

Index(['1983', '1984', '1985', '1986', '1987', '1988', '1989', '1992', '1993',
       '1994', '1995', '1999', '2000', '2002', '2005', '2007', '2008', '2014',
       '2016', '2020'],
      dtype='object')
Index(['1985', '1992', '1993', '1995', '1999', '2000', '2005', '2016'], dtype='object')


In [ ]:
# morocco_cities[morocco_cities["city_name"] == "Beni Mellal"]

,lat,lon,city_name
19,32.3394,-6.3608,Beni Mellal


In [26]:
estimator = len([item for item in dict_prec_dominence.values() if item]) / len(dict_prec_dominence)

In [36]:
from scipy import stats

z = np.sqrt(52) * (estimator - 0.9) / np.sqrt(0.9 * 0.1)

stats.norm.sf(z)

0.9999999999998599